In [0]:
import pytest
from sendData import run
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
import json
import random
import datetime

# Mock de EventHubProducerClient pour éviter de se connecter à Azure Event Hub pendant les tests
class MockEventHubProducerClient:
    def __init__(self, conn_str, eventhub_name):
        self.conn_str = conn_str
        self.eventhub_name = eventhub_name

    async def __aenter__(self):
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        pass

    async def create_batch(self):
        return MockEventDataBatch()

class MockEventDataBatch:
    def __init__(self):
        self.events = []

    def add(self, event):
        self.events.append(event)

    def __len__(self):
        return len(self.events)

# Test pour vérifier que les données générées sont valides
@pytest.mark.asyncio
async def test_run():
    # Mock de EventHubProducerClient
    producer = MockEventHubProducerClient("mock-connection-string", "mock-eventhub-name")

    # Exécuter la fonction run avec le mock
    await run()

    # Vérifier que les données générées sont valides
    async with producer:
        for i in range(400):
            event_data_batch = await producer.create_batch()

            # Simuler une transaction client
            transaction = {
                "transactionID": random.randint(1000, 9999),
                "customerID": random.randint(1, 100),
                "amount": round(random.uniform(10.0, 500.0), 2),
                "timestamp": datetime.datetime.now().isoformat(),
                "lastTransactionDate": datetime.datetime(random.randint(2019, 2024), random.randint(1, 12), random.randint(1, 28)).date().isoformat(),
                "type": "Achat vêtements",
                "customerEmail": f"customer{random.randint(1, 100)}@{random.choice(['gmail.com', 'outlook.com', 'live.com', 'apple.com', 'hotmail.com', 'yahoo.fr'])}",
                "customerFirstName": random.choice([
                    "John", "Jane", "Alex", "Emily", "Chris", "Michael", "Sarah", "David", "Laura", "Daniel",
                    "Emma", "James", "Olivia", "Matthew", "Sophia", "Joshua", "Isabella", "Andrew", "Mia", "Joseph",
                    "Charlotte", "Benjamin", "Amelia", "Samuel", "Harper", "William", "Evelyn", "Elijah", "Abigail",
                    "Mason", "Ella", "Logan", "Avery", "Lucas", "Scarlett", "Henry", "Grace", "Alexander", "Chloe",
                    "Sebastian", "Lily", "Jack", "Aria", "Owen", "Zoe", "Gabriel", "Nora", "Carter", "Lillian"
                ]),
                "customerLastName": random.choice([
                    "Doe", "Smith", "Johnson", "Williams", "Brown", "Taylor", "Anderson", "Thomas", "Jackson", "White",
                    "Harris", "Martin", "Thompson", "Garcia", "Martinez", "Robinson", "Clark", "Rodriguez", "Lewis", "Lee",
                    "Walker", "Hall", "Allen", "Young", "Hernandez", "King", "Wright", "Lopez", "Hill", "Scott",
                    "Green", "Adams", "Baker", "Gonzalez", "Nelson", "Carter", "Mitchell", "Perez", "Roberts",
                    "Schmidt", "Müller", "Schneider", "Fischer", "Weber", "Meyer", "Wagner", "Becker", "Hoffmann", "Schulz"
                ])
            }

            # Simuler un log de serveur
            log_entry = {
                "logID": random.randint(1, 1000),
                "status": random.choice(["INFO", "ERROR", "WARNING"]),
                "message": f"Processed transaction {transaction['transactionID']}",
                "logDate": transaction["lastTransactionDate"]
            }

            # Simuler un post sur les réseaux sociaux
            social_media_post = {
                "postID": random.randint(1, 5000),
                "customerEmail": transaction["customerEmail"],
                "content": random.choice([
                    "Just bought something amazing!",
                    "Not satisfied with the purchase.",
                    "Great customer service!",
                    "Will definitely buy again.",
                    "Terrible experience, not recommended."
                ]),
                "postDate": transaction["lastTransactionDate"],
                "pseudonym": f"Lolo{transaction['customerFirstName']}_{transaction['customerLastName']}Umami85"
            }

            # Combiner tous les événements en une structure JSON unique
            combined_event = {**transaction, **log_entry, **social_media_post}

            # Ajouter l'événement combiné au lot(batch)
            event_data_batch.add(EventData(json.dumps(combined_event)))

            # Vérifier que l'événement a été ajouté au lot
            assert len(event_data_batch) == 1